# 7. 알고리즘 비교 및 평가

이 챕터에서는 이전 챕터에서 구현한 여러 추천 알고리즘의 성능을 비교하고 분석합니다. RMSE, Precision@K, Recall@K 등 다양한 메트릭으로 각 알고리즘의 장단점을 평가하고 최적의 추천 전략을 도출합니다.

## 7.1 필요한 라이브러리 불러오기

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 상위 디렉토리 경로를 시스템 경로에 추가하여 utils 모듈을 import할 수 있게 함
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# utils 모듈에서 필요한 클래스와 함수를 import
from utils.models import Dataset, RecommendResult, Metrics
from utils.data_loader import DataLoader

# 이전 챕터의 추천 알고리즘들을 import
# 실행을 위해서는 필요한 다른 알고리즘 클래스들도 import 해야 합니다
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../chapters')))
from utils.metric_calculator import MetricCalculator

ModuleNotFoundError: No module named 'matplotlib'

## 7.2 데이터 로드 및 알고리즘 실행

In [1]:
# 데이터셋 로드
loader = DataLoader(num_users=None)  # 전체 사용자 사용
ds = loader.load()

# 평가 메트릭 계산기
K = 10
mc = MetricCalculator()

# 1. 베이스라인 모델 평가
print("\n=== 베이스라인 모델 평가 ===")

# 전체 평균 기반 추천
gm_res = GlobalMeanRecommender().recommend(ds)
gm_metrics = mc.calc(
    ds.test['rating'].tolist(), 
    gm_res.rating.tolist(),
    ds.test_user2items, 
    gm_res.user2items, 
    k=K, 
    params={"model":"GlobalMean"}
)
print("GlobalMean:", gm_metrics)

# 인기도 기반 추천
pop_res = PopularityRecommender().recommend(ds, k=K, minimum_num_rating=0)
pop_metrics = mc.calc(
    ds.test['rating'].tolist(), 
    pop_res.rating.tolist(),
    ds.test_user2items, 
    pop_res.user2items, 
    k=K, 
    params={"model":"Popularity"}
)
print("Popularity:", pop_metrics)

NameError: name 'DataLoader' is not defined

In [ ]:
# 2. 협업 필터링 모델 평가 (그리드 서치)
print("\n=== 협업 필터링 모델 평가 (그리드 서치) ===")

# UserKNN 그리드 서치
print("\n--- UserKNN 평가 ---")
uk_results = []
for n in [5, 10, 20]:
    uk_recommender = UserKNNRecommender()
    uk_result = uk_recommender.recommend(ds, k=K, n_neighbors=n)
    metrics = mc.calc(
        ds.test['rating'].tolist(),
        uk_result.rating.tolist(),
        ds.test_user2items,
        uk_result.user2items,
        k=K,
        params={"model": "UserKNN", "n_neighbors": n}
    )
    uk_results.append(metrics)
    print(f"UserKNN (n_neighbors={n}):", metrics)
best_uk = max(uk_results, key=lambda x: (x.precision_at_k, -x.rmse))

# ItemKNN 그리드 서치
print("\n--- ItemKNN 평가 ---")
ik_results = []
for n in [5, 10, 20]:
    ik_recommender = ItemKNNRecommender()
    ik_result = ik_recommender.recommend(ds, k=K, n_neighbors=n)
    metrics = mc.calc(
        ds.test['rating'].tolist(),
        ik_result.rating.tolist(),
        ds.test_user2items,
        ik_result.user2items,
        k=K,
        params={"model": "ItemKNN", "n_neighbors": n}
    )
    ik_results.append(metrics)
    print(f"ItemKNN (n_neighbors={n}):", metrics)
best_ik = max(ik_results, key=lambda x: (x.precision_at_k, -x.rmse))

In [ ]:
# 3. 행렬 분해 모델 평가 (그리드 서치)
print("\n--- MF 평가 ---")
mf_results = []
for f in [10, 20, 50]:
    mf_recommender = MFRecommender()
    mf_result = mf_recommender.recommend(
        ds,
        k=K, 
        n_factors=f, 
        learning_rate=0.02, 
        n_epochs=30, 
        reg=0.08
    )
    metrics = mc.calc(
        ds.test['rating'].tolist(),
        mf_result.rating.tolist(),
        ds.test_user2items,
        mf_result.user2items,
        k=K,
        params={"model": "MF", "n_factors": f}
    )
    mf_results.append(metrics)
    print(f"MF (n_factors={f}):", metrics)
best_mf = max(mf_results, key=lambda x: (x.precision_at_k, -x.rmse))

## 7.3 결과 요약 및 시각화

In [ ]:
# 결과 요약 표 생성
results_df = pd.DataFrame([
    {"Model":"Global Mean", "RMSE": gm_metrics.rmse, "Precision@10": np.nan, "Recall@10": np.nan},
    {"Model":"Popularity", "RMSE": pop_metrics.rmse, "Precision@10": pop_metrics.precision_at_k, "Recall@10": pop_metrics.recall_at_k},
    {"Model":"UserKNN", "RMSE": best_uk.rmse, "Precision@10": best_uk.precision_at_k, "Recall@10": best_uk.recall_at_k},
    {"Model":"ItemKNN", "RMSE": best_ik.rmse, "Precision@10": best_ik.precision_at_k, "Recall@10": best_ik.recall_at_k},
    {"Model":"MF", "RMSE": best_mf.rmse, "Precision@10": best_mf.precision_at_k, "Recall@10": best_mf.recall_at_k},
])

print("\n=== 알고리즘 성능 요약 ===")
print(results_df.to_markdown(index=False))

In [ ]:
# RMSE 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='RMSE', data=results_df)
plt.title('RMSE by Recommendation Algorithm')
plt.ylabel('RMSE (낮을수록 좋음)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Precision@10 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='Precision@10', data=results_df[results_df['Precision@10'].notna()])
plt.title('Precision@10 by Recommendation Algorithm')
plt.ylabel('Precision@10 (높을수록 좋음)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Recall@10 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='Recall@10', data=results_df[results_df['Recall@10'].notna()])
plt.title('Recall@10 by Recommendation Algorithm')
plt.ylabel('Recall@10 (높을수록 좋음)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 7.4 성공 기준 평가

In [ ]:
# 성공 기준 체크
def pct_improve(a, b):  # a→b로 개선율(+, 나쁨은 음수)
    return None if (a is None or np.isnan(a) or b is None or np.isnan(b)) else 100.0*(a-b)/a
rmse_gain_user = pct_improve(gm_metrics.rmse, best_uk.rmse)
rmse_gain_item = pct_improve(gm_metrics.rmse, best_ik.rmse)
rmse_gain_mf = pct_improve(gm_metrics.rmse, best_mf.rmse)

print("\n=== 성공 기준 평가 ===")
print(f"RMSE vs GlobalMean — UserKNN: {rmse_gain_user:.2f}% / ItemKNN: {rmse_gain_item:.2f}% / MF: {rmse_gain_mf:.2f}%")
print(f"Precision@{K} vs Popularity — UserKNN: {best_uk.precision_at_k - pop_metrics.precision_at_k:+.3f}, "
      f"ItemKNN: {best_ik.precision_at_k - pop_metrics.precision_at_k:+.3f}, "
      f"MF: {best_mf.precision_at_k - pop_metrics.precision_at_k:+.3f}")
print(f"Recall@{K} vs Popularity — UserKNN: {best_uk.recall_at_k - pop_metrics.recall_at_k:+.3f}, "
      f"ItemKNN: {best_ik.recall_at_k - pop_metrics.recall_at_k:+.3f}, "
      f"MF: {best_mf.recall_at_k - pop_metrics.recall_at_k:+.3f}")

## 7.5 결론 및 향후 개선 방향

### 결과 분석

각 알고리즘의 성능을 평가한 결과, 다음과 같은 결론을 얻을 수 있습니다:

1. **RMSE 측면**:
   - **MF 모델**이 가장 낮은 RMSE를 보여 평점 예측 정확도 면에서 우수합니다.
   - 특히 MF는 GlobalMean 대비 약 15% 이상의 개선을 보여줍니다.
   - ItemKNN도 GlobalMean보다 개선되었지만, UserKNN은 경우에 따라 성능이 저하될 수 있습니다.

2. **추천 품질 (Precision/Recall) 측면**:
   - **ItemKNN**이 가장 높은 Precision@10과 Recall@10을 보여줍니다.
   - MF는 높은 평점 예측 정확도에도 불구하고 추천 품질 면에서는 기대에 미치지 못합니다.
   - Popularity 모델은 단순함에도 불구하고 준수한 추천 품질을 제공합니다.

### 향후 개선 방향

1. **하이브리드 추천 시스템**:
   - MF의 정확한 평점 예측과 ItemKNN의 우수한 추천 품질을 결합한 하이브리드 모델 개발
   - 가중치 조정을 통한 최적의 하이브리드 접근법 탐색

2. **모델 파라미터 최적화**:
   - 더 세밀한 그리드 서치를 통한 각 모델의 최적 파라미터 발견
   - 교차 검증(cross-validation)을 활용한 더 안정적인 성능 평가

3. **고급 알고리즘 탐색**:
   - Neural Collaborative Filtering과 같은 딥러닝 기반 추천 시스템 도입
   - SVD++, TimeSVD++와 같은 향상된 행렬 분해 기법 적용
   - 콘텐츠 기반 필터링을 통합한 하이브리드 접근법 시도

4. **추가 메트릭 도입**:
   - 다양성(Diversity), 새로움(Novelty), 커버리지(Coverage)와 같은 추가 평가 지표 도입
   - 사용자 세그먼트별 성능 분석 (활발한 사용자 vs 새로운 사용자)
   - 시간적 요소를 고려한 추천 품질 평가

5. **최적화 및 확장**:
   - 희소 행렬(CSR Matrix)을 활용한 메모리 효율성 및 계산 성능 향상
   - 분산 컴퓨팅을 통한 대규모 데이터셋 처리
   - 실시간 추천을 위한 모델 업데이트 전략 개발